# Code Report

## Primary imports and functions

### Imports and print_grid, check_exist function and test_cases name

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from   IPython.core.display import display, HTML
def print_grid(data):
    cells = np.zeros((8,8))
    for item in data:
        cells[item[0]-1][item[1]-1] = 1
    for row in cells:
        print(" ".join(["X" if item == 1 else "O" for item in row]))
def check_exist(data,x,y):
    for i in range(8):
        if data[i][0] == x and data[i][1] == y:
            return False
    return True
def print_as_header(input):
    display(HTML('<h1>'+input+'</h1>'))

### Check The State Of Grid

In [2]:
def check_mutual_cells(data,column_number):
    counter = 0
    que     = []
    for item in data:
        if item[column_number] in que:
            counter += 1
        else:
            que.append(item[column_number])
    return counter
def count_row_threats(data):
    return check_mutual_cells(data,0)
def count_column_threats(data):
    return check_mutual_cells(data,1)
def count_diagonal_threats(data):
    counter = 0
    cells = np.zeros((8,8))
    for item in data:
        x = item[0]-1
        y = item[1]-1
        if cells[x][y] == 1:
            counter += 1
        for i in range(-8,8):
            if 0<=x+i<=7:
                if 0<=y+i<=7:
                    cells[x+i][y+i] = 1
                if 0<=y-i<=7:
                    cells[x+i][y-i] = 1
    return counter
def count_all(data):
    return count_row_threats(data) + count_column_threats(data) + count_diagonal_threats(data)
def check_condtions(data):
    if count_row_threats(data)==0:
        if count_column_threats(data)==0:
            if count_diagonal_threats(data)==0:
                return True
    return False

### move Function That Is Used In IDFS, BFS, A*

In [3]:
def sub_move(data,index,direction,diff_x,diff_y,mulitply):
    if direction == 0 or direction == 8:
        diff_x = -mulitply
    elif direction == 2:
        diff_y = mulitply
    elif direction == 4:
        diff_x = mulitply
    else:
        diff_y = -mulitply
    return diff_x,diff_y
def base_move(data,index,direction,mulitply):
    diff_x = 0
    diff_y = 0
    if direction%2 == 0:
        diff_x,diff_y = sub_move(data,index,direction,diff_x,diff_y,mulitply)
    else:
        diff_x,diff_y = sub_move(data,index,direction-1,diff_x,diff_y,mulitply)
        diff_x,diff_y = sub_move(data,index,direction+1,diff_x,diff_y,mulitply)
    new_x = data[index][0] + diff_x
    new_y = data[index][1] + diff_y
    return new_x,new_y
def pure_move(data,index,direction,mulitply=1):
    new_x,new_y = base_move(data,index,direction,mulitply)
    data[index][0] = new_x
    data[index][1] = new_y
def move(data,index,direction,mulitply=1):
    new_x,new_y = base_move(data,index,direction,mulitply)
    if 0<new_x<9 and 0<new_y<9:
        if check_exist(data,new_x,new_y):
            data[index][0] = new_x
            data[index][1] = new_y
            return True
    return False

### Testcase File Names

In [4]:
test_cases1 = ["in1.csv","in2.csv","in3.csv"]
test_cases2 = ["test_a.csv","test_b.csv","test_c.csv"]
idfs_test = test_cases2
bfs_test = test_cases2[0:2]
a_star_test = test_cases1 + test_cases2

### super_move Functions Used In Improved IDFS

In [5]:
def base_super_move(data,index,direction,mulitply):
    length = 2*abs(mulitply)
    other = direction%length - abs(mulitply)
    if mulitply<0:
        other = -other
    if direction < length:
        diff_x = other
        diff_y = mulitply
    elif direction < 2*length:
        diff_x = mulitply
        diff_y = -other
    elif direction < 3*length:
        diff_x = -other
        diff_y = -mulitply
    else:
        diff_x = -mulitply
        diff_y = other
    new_x = data[index][0] + diff_x
    new_y = data[index][1] + diff_y
    return new_x,new_y
def pure_super_move(data,index,direction,mulitply=1):
    new_x,new_y = base_super_move(data,index,direction,mulitply)
    data[index][0] = new_x
    data[index][1] = new_y
def super_move(data,index,direction,mulitply=1):
    new_x,new_y = base_super_move(data,index,direction,mulitply)
    if 0<new_x<9 and 0<new_y<9:
        if check_exist(data,new_x,new_y):
            data[index][0] = new_x
            data[index][1] = new_y
            return True
    return False

# IDFS

In [6]:
total_moves = 0
def idfs(data,level):
    if level == 0:
        if check_condtions(data):
            return data
        else:
            return None
    for index in range(8):
        for direction in range(8):
            if move(data,index,direction):
                global total_moves
                total_moves += 1
                ret = idfs(data,level-1)
                if ret is not None:
                    return ret
                pure_move(data,index,direction,-1)
total_time = 0
for file_name in idfs_test:
    data = pd.read_csv(file_name,header=None)
    data = data.values.tolist()
    begin = time.time()
    for i in range(0,8):
        ret = idfs(data,i)
        if ret is not None:
            end = time.time()
            print_as_header(file_name)
            print_grid(ret)
            print ("The number of moves to correct answer is "+str(i))
            print ("The number of total moves is "+str(total_moves))
            break
    print ("The time is " + str(end-begin))
    total_time+=end-begin
print ("The avereage time is " + str(total_time/len(idfs_test)))

O X O O O O O O
O O O X O O O O
O O O O O X O O
O O O O O O O X
O O X O O O O O
X O O O O O O O
O O O O O O X O
O O O O X O O O
The number of moves to correct answer is 3
The number of total moves is 69045
The time is 0.2503700256347656


O X O O O O O O
O O O X O O O O
O O O O O X O O
O O O O O O O X
O O X O O O O O
X O O O O O O O
O O O O O O X O
O O O O X O O O
The number of moves to correct answer is 4
The number of total moves is 4608473
The time is 16.01227903366089


KeyboardInterrupt: 

# Improved IDFS

### Improved IDFS Requirement Functions

In [ ]:
def sub_increament(numbers,maxi,i):
    numbers[-i] += 1
    if numbers[-i] == maxi-i:
        sub_increament(numbers,maxi,i+1)
        numbers[-i] = numbers[-(i+1)] + 1
def increament(numbers,maxi):
    try:
        sub_increament(numbers,maxi+1,1)
        return True
    except:
        return False
def create_list(begin,end):
    return [i for i in range(begin,end)]
def convert_to_move(numbers,length):
    arr = [0]*length
    for n in numbers:
        arr[n] = 1
    temp = 0
    ret  = []
    for i in arr:
        if i == 0:
            ret.append(temp)
            temp = 0
        else:
            temp+=1
    ret.append(temp)
    return ret

### Improved IDFS Main Functions

In [ ]:
def sub_improved_idfs(data,moves,index):
    next_index = 0
    for next_index in range(index+1,9):
        if next_index == 8:
            break
        if moves[next_index] != 0:
            break
    for direction in range(8*moves[index]):
        if super_move(data,index,direction,moves[index]):
            if next_index == 8:
                if check_condtions(data):
                    return data
            else:
                ret = sub_improved_idfs(data,moves,next_index)
                if ret is not None:
                    return ret
            pure_super_move(data,index,direction,-moves[index])
    return None
def improved_idfs(data,level):
    total_moves = 0
    if level == 0:
        if check_condtions(data):
            return data
        else:
            return None
    n = create_list(0,level)
    while True:
        moves = convert_to_move(n,7+level)
        total_moves += level
        ret = sub_improved_idfs(data,moves,n[0])
        if ret is not None:
            return total_moves,ret
        if not increament(n,7+level):
            break
    return None
# total_time = 0
# for file_name in a_star_test:
#     data = pd.read_csv(file_name,header=None)
#     data = data.values.tolist()
#     begin = time.time()
#     for i in range(0,8):
#         ret = improved_idfs(data,i)
#         if ret is not None:
#             end = time.time()
#             print_as_header(file_name)
#             print_grid(ret[1])
#             print ("The number of moves to correct answer is "+str(i))
#             print ("The number of total moves is "+str(ret[0]))
#             break
#     print ("The time is " + str(end-begin))
#     total_time+=end-begin
# print ("The avereage time is " + str(total_time/ln(a_star_test)))

# BFS

In [ ]:
from collections import deque
total_moves = 0
def my_hash(data):
    ret = ''
    for row in data:
        ret += chr(row[0]*8+row[1])
    return ret
def bfs(data):
    visited = {}
    should_visit = deque()
    move_count = deque()
    should_visit.append(data)
    move_count.append(0)
    while True:
        data = should_visit.popleft()
        curent_move_count = move_count.popleft()
        if my_hash(data) not in visited:
            global total_moves
            total_moves += 1
            if check_condtions(data):
                return curent_move_count,data
            visited[my_hash(data)] = 0
            for index in range(8):
                for direction in range(8):
                    new_d = [row[:] for row in data]
                    if move(new_d,index,direction):
                        if my_hash(new_d) not in visited:
                            should_visit.append(new_d)
                            move_count.append(curent_move_count+1)
total_time = 0
for file_name in bfs_test:
    data = pd.read_csv(file_name,header=None)
    data = data.values.tolist()
    begin = time.time()
    ret = bfs(data)
    print_as_header(file_name)
    end = time.time()
    print_grid(ret[1])
    print ("The number of moves to correct answer is "+str(ret[0]))
    print ("The number of total moves is "+str(total_moves))
    print ("The time is " + str(end-begin))
    total_time+=end-begin
print ("The avereage time without third file is " + str(total_time/len(bfs_test)))

# A*

In [ ]:
def a_star(data):
    pop_count = 0
    visited = {}
    should_visit = []
    grades = []
    move_count = []
    should_visit.append(data)
    grades.append(count_all(data))
    move_count.append(0)
    while True:
        ind = grades.index(min(grades))
        data = should_visit.pop(ind)
        grades.pop(ind)
        curent_move_count = move_count.pop(ind)
        pop_count += 1
        if my_hash(data) not in visited:
            visited[my_hash(data)] = 0
            for index in range(8):
                for direction in range(8):
                    new_d = [row[:] for row in data]
                    if move(new_d,index,direction):
                        if my_hash(new_d) not in visited:
                            h = count_all(new_d)
                            if h == 0:
                                return curent_move_count+1,pop_count,new_d
                            should_visit.append(new_d)
                            grades.append(h+1.2*curent_move_count)
                            move_count.append(curent_move_count+1)
total_time = 0
for file_name in a_star_test:
    data = pd.read_csv(file_name,header=None)
    data = data.values.tolist()
    begin = time.time()
    ret = a_star(data)
    end = time.time()
    print_as_header(file_name)
    print_grid(ret[2])
    print ("The number of moves to correct answer is "+str(ret[0]))
    print ("The number of total moves is "+str(ret[1]))
    print ("The time is " + str(end-begin))
    total_time+=end-begin
print ("The avereage time is " + str(total_time/len(a_star_test)))